In [14]:
# Import chunk library + Dataset

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import unicodecsv
import seaborn as sns

ways_nodes = pd.read_csv("~/PycharmProjects/DAND_OpenStreeMap/subset/ways_nodes.csv")
ways       = pd.read_csv("~/PycharmProjects/DAND_OpenStreeMap/subset/ways.csv")
ways_tags  = pd.read_csv("~/PycharmProjects/DAND_OpenStreeMap/subset/ways_tags.csv")
nodes      = pd.read_csv("~/PycharmProjects/DAND_OpenStreeMap/subset/nodes.csv")
nodes_tags = pd.read_csv("~/PycharmProjects/DAND_OpenStreeMap/subset/nodes_tags.csv")


### Way Nodes :

In [8]:
print ways_nodes.columns
print len(ways_nodes)

Index([u'id', u'node_id', u'position'], dtype='object')
12242


In [9]:
ways_nodes.head()

,id,node_id,position
0,4803849,34691601,0
1,4803849,1801000290,1
2,4803849,1164867374,2
3,4803849,3397409847,3
4,4803849,1937688909,4


### Nodes :

In [16]:
print ways.columns
print len(ways)

Index([u'id', u'user', u'uid', u'version', u'changeset', u'timestamp'], dtype='object')
1410


In [17]:
ways.head()

,id,user,uid,version,changeset,timestamp
0,4803849,Martin Borsje,99810,35,37882240,2016-03-16T20:27:25Z
1,4803852,yxcv,26203,17,49958610,2017-07-01T06:20:06Z
2,11963069,yu nus,6423151,16,50938057,2017-08-08T09:03:50Z
3,11963253,yu nus,6423151,20,50938057,2017-08-08T09:03:51Z
4,11963816,Sebra1177,2557264,12,28099349,2015-01-12T22:15:44Z


### Way Nodes :

In [ ]:
print ways_tags.columns
print len(ways_tags)

In [18]:
ways_tags.head()

,id,key,value,type
0,4803849,highway,primary,regular
1,4803849,lanes,2,regular
2,4803849,maxspeed,50,regular
3,4803849,name,Nauenstrasse,regular
4,4803849,oneway,yes,regular


In [ ]:
### Way Nodes :